In [1]:
import pandas as pd
from sqlalchemy import create_engine
import matplotlib.pyplot as plt
from config import postgresPass as pw

In [2]:
csv_file = "csv/2022.csv"
happiness_df = pd.read_csv(csv_file)
happiness_df = happiness_df[['Country', 'RANK']]
happiness_df = happiness_df.rename(columns={"RANK": 'happiness_rank', 'Country':'country'})
happiness_df

,country,happiness_rank
0,Finland,1
1,Denmark,2
2,Iceland,3
3,Switzerland,4
4,Netherlands,5
...,...,...
142,Rwanda*,143
143,Zimbabwe,144
144,Lebanon,145
145,Afghanistan,146


In [3]:
url = 'https://en.wikipedia.org/wiki/List_of_countries_and_dependencies_by_population_density'
tables = pd.read_html(url)
pop_dens = tables[0]
country_list = pop_dens['Country (or territory)']['Country (or territory)']
country_list

0                      Macau * (China)
1                             Monaco *
2                          Singapore *
3                  Hong Kong * (China)
4                    Gibraltar * (BOT)
                    ...               
245         Western Sahara * [note 11]
246                         Mongolia *
247           Falkland Islands * (BOT)
248    Svalbard and Jan Mayen (Norway)
249              Greenland * (Denmark)
Name: Country (or territory), Length: 250, dtype: object

In [4]:
NEW_country_list = []
for country in country_list:
    NEW_country_list.append(country.partition('\u202f*')[0])
NEW_country_list

['Macau',
 'Monaco',
 'Singapore',
 'Hong Kong',
 'Gibraltar',
 'Bahrain',
 'Malta',
 'Maldives',
 'Bermuda',
 'Sint Maarten',
 'Bangladesh',
 'Guernsey',
 'Vatican City',
 'Jersey',
 'Palestine',
 'Mayotte',
 'Lebanon',
 'Barbados',
 'Saint Martin (France)',
 'Taiwan',
 'Mauritius',
 'Aruba',
 'San Marino',
 'Nauru',
 'South Korea',
 'Saint Barthélemy',
 'Rwanda',
 'Comoros',
 'Netherlands',
 'Israel',
 'Haiti',
 'India',
 'Burundi',
 'Tuvalu',
 'Belgium',
 'Philippines',
 'Curaçao',
 'Puerto Rico',
 'Réunion',
 'Sri Lanka',
 'Japan',
 'Martinique',
 'Guam',
 'El Salvador',
 'Grenada',
 'Marshall Islands',
 'U.S. Virgin Islands',
 'Vietnam',
 'Saint Lucia',
 'American Samoa',
 'Saint Vincent and the Grenadines',
 'United Kingdom',
 'Pakistan',
 'Trinidad and Tobago',
 'Cayman Islands',
 'Jamaica',
 'Kuwait',
 'Liechtenstein',
 'Guadeloupe',
 'Luxembourg',
 'Qatar',
 'Antigua and Barbuda',
 'Germany',
 'Nigeria',
 'Dominican Republic',
 'British Virgin Islands',
 'Seychelles',
 'North 

In [5]:
type(NEW_country_list)

list

In [6]:
km2pd = pop_dens['Density']['/km2']
type(km2pd)

pandas.core.series.Series

In [7]:
popdensDF = pd.concat([pd.Series(NEW_country_list), km2pd], axis = 1)
popdensDF=popdensDF.rename(columns = {0: 'Country'})
popdensDF

,Country,/km2
0,Macau,21055.00
1,Monaco,19341.00
2,Singapore,8041.00
3,Hong Kong,6677.00
4,Gibraltar,5620.00
...,...,...
245,Western Sahara,2.00
246,Mongolia,2.00
247,Falkland Islands,0.30
248,Svalbard and Jan Mayen (Norway),0.04


In [8]:
popdensDF = popdensDF.rename(columns ={"/km2": "p_dense", 'Country': 'country'})
popdensDF

,country,p_dense
0,Macau,21055.00
1,Monaco,19341.00
2,Singapore,8041.00
3,Hong Kong,6677.00
4,Gibraltar,5620.00
...,...,...
245,Western Sahara,2.00
246,Mongolia,2.00
247,Falkland Islands,0.30
248,Svalbard and Jan Mayen (Norway),0.04


In [9]:
protocol = 'postgresql'
username = 'postgres'
password = pw
host = 'localhost'
port = 5432
database_name = 'happy density'
rds_connection_string = f'{protocol}://{username}:{password}@{host}:{port}/{database_name}'
engine = create_engine(rds_connection_string)

In [10]:
engine.table_names()

C:\Users\David\AppData\Local\Temp/ipykernel_13456/4162273999.py:1: SADeprecationWarning: The Engine.table_names() method is deprecated and will be removed in a future release.  Please refer to Inspector.get_table_names(). (deprecated since: 1.4)
  engine.table_names()


['happiness', 'pop_dense']

In [11]:
popdensDF.to_sql(name='pop_dense', con=engine, if_exists='append', index=False)

IntegrityError: (psycopg2.errors.UniqueViolation) duplicate key value violates unique constraint "pop_dense_pkey"
DETAIL:  Key (country)=(Macau) already exists.

[SQL: INSERT INTO pop_dense (country, p_dense) VALUES (%(country)s, %(p_dense)s)]
[parameters: ({'country': 'Macau', 'p_dense': 21055.0}, {'country': 'Monaco', 'p_dense': 19341.0}, {'country': 'Singapore', 'p_dense': 8041.0}, {'country': 'Hong Kong', 'p_dense': 6677.0}, {'country': 'Gibraltar', 'p_dense': 5620.0}, {'country': 'Bahrain', 'p_dense': 1913.0}, {'country': 'Malta', 'p_dense': 1390.0}, {'country': 'Maldives', 'p_dense': 1719.0}  ... displaying 10 of 250 total bound parameter sets ...  {'country': 'Svalbard and Jan Mayen (Norway)', 'p_dense': 0.04}, {'country': 'Greenland', 'p_dense': 0.03})]
(Background on this error at: https://sqlalche.me/e/14/gkpj)

In [ ]:
happiness_df.to_sql(name='happiness', con=engine, if_exists='append', index=False)

## SQL code
create table happiness(
	country text primary key,
	happiness_rank int
);

create table pop_dense(
	country text primary key,
	p_dense int
);

select happiness.country,
	happiness.happiness_rank,
	pop_dense.p_dense
	from happiness
	inner JOIN pop_dense
	ON happiness.country = pop_dense.country;